# Model Deployment using PySpark

In this notebook we are going to deploy as microservice the model built in the previous [notebook](./model.ipynb).

## Load libraries and data


In [1]:
from mleap import pyspark
from pyspark.ml.linalg import Vectors
from mleap.pyspark.spark_support import SimpleSparkSerializer
from pyspark.sql import functions as f
from pyspark.ml.feature import VectorAssembler, MaxAbsScaler, OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import NaiveBayes

In [17]:
file_path = "/sparta/dibesa/german_credit_data_labels.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

Remove unusable columns

In [18]:
df = df.drop('Sex', 'Job')
df.show()

+---+---+-------+---------------+----------------+-------------+--------+-------------------+----+
| ID|Age|Housing|Saving accounts|Checking account|Credit amount|Duration|            Purpose|Risk|
+---+---+-------+---------------+----------------+-------------+--------+-------------------+----+
|  0| 67|    own|             NA|          little|         1169|       6|           radio/TV|good|
|  1| 22|    own|         little|        moderate|         5951|      48|           radio/TV| bad|
|  2| 49|    own|         little|              NA|         2096|      12|          education|good|
|  3| 45|   free|         little|          little|         7882|      42|furniture/equipment|good|
|  4| 53|   free|         little|          little|         4870|      24|                car| bad|
|  5| 35|   free|             NA|              NA|         9055|      36|          education|good|
|  6| 53|    own|     quite rich|              NA|         2835|      24|furniture/equipment|good|
|  7| 35| 

In [19]:
encoder = StringIndexer(inputCol='Risk', outputCol='Binary_Risk')
df = encoder.fit(df).transform(df)
df = df.drop('Risk')
df.show()

+---+---+-------+---------------+----------------+-------------+--------+-------------------+-----------+
| ID|Age|Housing|Saving accounts|Checking account|Credit amount|Duration|            Purpose|Binary_Risk|
+---+---+-------+---------------+----------------+-------------+--------+-------------------+-----------+
|  0| 67|    own|             NA|          little|         1169|       6|           radio/TV|        0.0|
|  1| 22|    own|         little|        moderate|         5951|      48|           radio/TV|        1.0|
|  2| 49|    own|         little|              NA|         2096|      12|          education|        0.0|
|  3| 45|   free|         little|          little|         7882|      42|furniture/equipment|        0.0|
|  4| 53|   free|         little|          little|         4870|      24|                car|        1.0|
|  5| 35|   free|             NA|              NA|         9055|      36|          education|        0.0|
|  6| 53|    own|     quite rich|             

## Transform data for our use case

In [20]:
df = df.withColumn('Checking_little', f.when(f.col('Checking account') == "little", 1).otherwise(0))
df = df.withColumn('Checking_null', f.when(f.col('Checking account') == "NA", 1.0).otherwise(0))
df = df.withColumn('Checking_moderate', f.when(f.col('Checking account') == "moderate", 1).otherwise(0))
df = df.withColumn('Savings_little', f.when(f.col('Saving accounts') == "little", 1).otherwise(0))
df = df.withColumn('Savings_null', f.when(f.col('Saving accounts') == "little", 1).otherwise(0))
df = df.withColumn('Purpose_radio/TV', f.when(f.col('Purpose') == "radio/TV", 1).otherwise(0))
df = df.withColumn('Housing_own', f.when(f.col('Housing') == "own", 1).otherwise(0))
df = df.withColumn('Credit_big', f.when(f.col('Credit amount') > 10000, 1).otherwise(0))
df = df.withColumn('Duration_short', f.when(f.col('Duration') < 12, 1).otherwise(0))
df = df.withColumn('Age_young', f.when(f.col('Age') < 27, 1).otherwise(0))

df = df.drop('Age', 'Housing', 'Saving accounts', 'Checking account', 'Credit amount', 'Duration', 'Purpose')
df.show()

+---+-----------+---------------+-------------+-----------------+--------------+------------+----------------+-----------+----------+--------------+---------+
| ID|Binary_Risk|Checking_little|Checking_null|Checking_moderate|Savings_little|Savings_null|Purpose_radio/TV|Housing_own|Credit_big|Duration_short|Age_young|
+---+-----------+---------------+-------------+-----------------+--------------+------------+----------------+-----------+----------+--------------+---------+
|  0|        0.0|              1|          0.0|                0|             0|           0|               1|          1|         0|             1|        0|
|  1|        1.0|              0|          0.0|                1|             1|           1|               1|          1|         0|             0|        1|
|  2|        0.0|              0|          1.0|                0|             1|           1|               0|          1|         0|             0|        0|
|  3|        0.0|              1|          0.0

# Define features

In [21]:
features = df.columns
features = features[2:]
features

['Checking_little',
 'Checking_null',
 'Checking_moderate',
 'Savings_little',
 'Savings_null',
 'Purpose_radio/TV',
 'Housing_own',
 'Credit_big',
 'Duration_short',
 'Age_young']

## Feature Pipeline

In [22]:
continuous_feature_assembler = VectorAssembler(inputCols=features, outputCol="all_features")

## Assemble our features and feature pipeline


In [23]:
featurePipeline = Pipeline(stages=[continuous_feature_assembler])

sparkFeaturePipelineModel = featurePipeline.fit(df)

print("Finished constructing the pipeline")

Finished constructing the pipeline


In [24]:
df.schema

StructType(List(StructField(ID,IntegerType,true),StructField(Binary_Risk,DoubleType,true),StructField(Checking_little,IntegerType,false),StructField(Checking_null,DoubleType,false),StructField(Checking_moderate,IntegerType,false),StructField(Savings_little,IntegerType,false),StructField(Savings_null,IntegerType,false),StructField(Purpose_radio/TV,IntegerType,false),StructField(Housing_own,IntegerType,false),StructField(Credit_big,IntegerType,false),StructField(Duration_short,IntegerType,false),StructField(Age_young,IntegerType,false)))

## Train a Naive Bayes Model

In [25]:
# Step 3.2 Create our model

nb = NaiveBayes(featuresCol="all_features", labelCol="Binary_Risk", smoothing=0.0001, modelType="multinomial")

pipeline_nb = [sparkFeaturePipelineModel] + [nb]

sparkPipelineEstimatornb = Pipeline(stages = pipeline_nb)

sparkPipelinenb = sparkPipelineEstimatornb.fit(df)

print("Complete: Training Naive Bayes")

Complete: Training Naive Bayes


In [26]:
%listPipelines

[
  {
    "modelName": "mnistlr4",
    "marathonState": {
      "tasksHealthy": 0,
      "tasksRunning": 0,
      "appStatus": "not_deployed",
      "deployments": [],
      "tasksStatus": [],
      "tasksStaged": 0,
      "tasksUnhealthy": 0
    },
    "versions": [
      {
        "framework": "spark",
        "user": "formacion1",
        "modelDescription": "Predict MNIST numbers using Logistic Regression",
        "files": [
          {
            "serializationLibVersion": "2.2.0.5",
            "type": "pipelinemodel",
            "serializationLib": "spark",
            "modelPath": "/intellmodelrep1/models/mnistlr4/v0/mnistlr4-spark-v0.zip"
          },
          {
            "serializationLibVersion": "0.7.0",
            "type": "pipelinemodel",
            "serializationLib": "mleap",
            "modelPath": "/intellmodelrep1/models/mnistlr4/v0/mnistlr4-mleap-v0.zip"
          }
        ],
        "additionalInfo": null,
        "timestamp": "2019-10-23 15:12:19.725",
  

## Save Models in HDFS

In [12]:
%savePipeline -h

usage: savePipeline [-h] [--pipelineName PIPELINENAME]
                    [--pipelineModelObject PIPELINEMODELOBJECT]
                    [--dataframe DATAFRAME] [--description DESCRIPTION]
                    [--additionalInfo ADDITIONALINFO]
                    [--serializationLib {mleap,spark}]

Serializes a PipelineModel to a zip file and upload it to the configured model
repository.

optional arguments:
  -h, --help            show this help message and exit
  --pipelineName PIPELINENAME
  --pipelineModelObject PIPELINEMODELOBJECT
  --dataframe DATAFRAME
  --description DESCRIPTION
  --additionalInfo ADDITIONALINFO
  --serializationLib {mleap,spark}


In [13]:
%deletePipeline -h

usage: deletePipeline [-h] [--pipelineName PIPELINENAME]

Deletes a PipelineModel stored in a model repository.

optional arguments:
  -h, --help            show this help message and exit
  --pipelineName PIPELINENAME


In [14]:
%deployPipeline -h

usage: deployPipeline [-h] [--pipelineName PIPELINENAME] [--cpus CPUS]
                      [--mem MEM] [--instances INSTANCES]

Deploy a PipelineModel stored in a model repository.

optional arguments:
  -h, --help            show this help message and exit
  --pipelineName PIPELINENAME
  --cpus CPUS           Number of cpus per deployed model microservice.
  --mem MEM             Memory per deployed model microservice instance (in
                        Megabytes).
  --instances INSTANCES
                        Number of instances that will be deployed.


In [29]:
%savePipeline   --pipelineName dibesa-model \
                --pipelineModelObject sparkPipelinenb \
                --dataframe df \
                --description "Naive Bayes model for predicting risk probability"

{"message":"Model 'dibesa-model' correctly uploaded."}


In [22]:
%deletePipeline --pipelineName dibesa-credit

Error while deleting 'airbnb' model from repository. Response from server:
 {"error":"ModelMetadataNotExistsException","reason":"Model 'airbnb' not found in the metadata repository. "}


In [30]:
%deployPipeline --pipelineName dibesa-model

Model 'dibesa-model' correctly deployed from repository.
